In [0]:
#Import Library
import numpy as np
import pandas as pd
from math import log
from sklearn import model_selection
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import accuracy_score as accuracy
import xgboost as xgb

In [0]:
#Import train data set
dataframe_training = pd.read_csv("train.csv")
y_train=dataframe_training['playtime_forever']
train_categories=dataframe_training['categories']
train_dummies_categories= train_categories.str.get_dummies(",")
train_genres=dataframe_training['genres']
train_dummies_genres= train_genres.str.get_dummies(",")
dataframe_training=dataframe_training.filter(items=['price','date_difference','total_positive_reviews','total_negative_reviews'])
x_train=pd.concat([dataframe_training,train_dummies_genres,train_dummies_categories], axis=1, sort=False)

In [0]:
#perform backward elimination
import statsmodels.api as sm
X_1 = sm.add_constant(x_train)
model_1= sm.OLS(y_train,X_1).fit()
model_1.pvalues


In [0]:
cols = list(x_train.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = x_train[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y_train,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


['date_difference', 'total_positive_reviews', 'total_negative_reviews', 'Co-op', 'Includes level editor', 'Local Multi-Player', 'Steam Workshop', 'Valve Anti-Cheat enabled']


In [0]:
#Variable Selection
x_train=x_train.filter(items=['date_difference', 'total_positive_reviews', 'total_negative_reviews', 'Adventure', 'RPG', 'Full controller support', 'Steam Workshop', 'price'])
x_train['reviews']=x_train['total_positive_reviews']+x_train['total_negative_reviews']
x_train=x_train.filter(items=['date_difference', 'reviews', 'Adventure', 'RPG', 'Full controller support', 'Steam Workshop', 'price'])

In [0]:
#Split data set into training data and testing data
Xtrain, Xtest, Ytrain, Ytest = split(x_train,y_train, test_size=0.25, random_state=7)

In [0]:
#Import Bagging Regressor
from sklearn.ensemble import *
AllRegressorModel = [BaggingRegressor]

In [0]:
def Model_Selection_By_Cross_Valid():
        ThisRound_SelectedModel = None
        ThisRound_SelectedModel_Name = None
        ThisRound_SelectedModel_Score = None
        for temp_select_model_name in AllRegressorModel:
            kfold = model_selection.KFold(n_splits=10, random_state=7)
            print (kfold)
            temp_model= temp_select_model_name()
            print(temp_model)
            temp_model.fit(Xtrain, Ytrain)
            results = model_selection.cross_val_score(temp_model, Xtrain, Ytrain, cv=kfold, scoring='neg_mean_squared_error')
        print(temp_select_model_name,results.mean())
        if (ThisRound_SelectedModel == None) or (abs(results.mean()) < ThisRound_SelectedModel_Score):
            ThisRound_SelectedModel = temp_model
            ThisRound_SelectedModel_Name = temp_select_model_name
            ThisRound_SelectedModel_Score = abs(results.mean())
        print ("This round Model Name: ", temp_model,"MSE Score: ",abs(results.mean()))
        print ("This Model Feature Importance",temp_model.feature_importances_)
        print("This Model Do No Have Feature Importance......")
        print ("<----------------------------------->")
        print ("Selected Model Name:", ThisRound_SelectedModel, "MSE Score:",ThisRound_SelectedModel_Score)
        return {"ModelName": ThisRound_SelectedModel_Name,"Model": ThisRound_SelectedModel}

In [0]:
SelectedModel = Model_Selection_By_Cross_Valid()


KFold(n_splits=10, random_state=7, shuffle=False)
BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=None, verbose=0,
                 warm_start=False)
<class 'sklearn.ensemble.bagging.BaggingRegressor'> -104.2676696174511
This round Model Name:  BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=None, verbose=0,
                 warm_start=False) MSE Score:  104.2676696174511


AttributeError: ignored

In [0]:
#Build model
predictmodel_new=BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=7, verbose=0,
                 warm_start=False) 

In [0]:
#Check model performance

In [0]:
predictmodel_new.fit(x_train,y_train)

BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=10,
                 n_jobs=None, oob_score=False, random_state=7, verbose=0,
                 warm_start=False)

In [0]:
y_train_predict=predictmodel_new.predict(x_train)

In [0]:
from sklearn.metrics import r2_score
r2_score(y_train, y_train_predict)

0.8655966995525417

In [0]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_train, y_train_predict)

1.5134827264044817

In [0]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train, y_train_predict)

16.851719700558817

In [0]:
#Import testing data set
dataframe_testing = pd.read_csv("test.csv")
test_categories=dataframe_testing['categories']
test_dummies_categories= test_categories.str.get_dummies(",")
test_genres=dataframe_testing['genres']
test_dummies_genres= test_genres.str.get_dummies(",")
dataframe_testing=dataframe_testing.filter(items=['price','date_difference','total_positive_reviews','total_negative_reviews'])
x_test=pd.concat([dataframe_testing,test_dummies_genres,test_dummies_categories], axis=1, sort=False)

In [0]:
#Elimination variables in testing data set
x_test=x_test.filter(items=['date_difference', 'total_positive_reviews', 'total_negative_reviews', 'Adventure', 'RPG', 'Full controller support', 'Steam Workshop', 'price'])
x_test['reviews']=x_test['total_positive_reviews']+x_test['total_negative_reviews']
x_test=x_test.filter(items=['date_difference', 'reviews', 'Adventure', 'RPG', 'Full controller support', 'Steam Workshop', 'price'])

In [0]:
x_test=x_test.filter(items=['date_difference','total_positive_reviews', 'total_negative_reviews','Adventure','RPG','Co-op ','Full controller support','Includes level editor ','Steam Workshop','price'])


In [0]:
#Predict the result

In [0]:
Predicted = predictmodel_new.predict(x_test)

In [0]:
#Export data in csv
Predicted=pd.DataFrame(Predicted)
from google.colab import files
Predicted.to_csv('Submission_88675.csv') 
files.download('Submission_88675.csv')